# A

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [6]:
#generate points for spline
def f(x):
    return 1/(1+25*x**2)

def df(x):
    return -50*x/(1+25*x**2)**2

for ii in range(0,5):
    N = 1+5*2**ii
    txt = open("output/A_"+str(N)+".txt",'w')
    txt.write(str(N)+"\n")
    x = np.linspace(-1,1,N)
    y = f(x)
    for i in range(0,N):
        txt.write(str(x[i])+" "+str(y[i])+"\n")
    txt.write("\n")
    txt.write(str(df(x[0]))+" "+str(df(x[N-1]))+"\n")
    txt.close()

In [ ]:
def eval_spline(x,points,coeffs):
    N = len(points)
    for i in range(0,N):
        if x>=points[i] and x<=points[i+1]:
            return coeffs[i,0]+coeffs[i,1]*(x-points[i])+coeffs[i,2]*(x-points[i])**2+coeffs[i,3]*(x-points[i])**3
    return 0

# get points and coefficients
for ii in range(0,5):
    N = 1+5*2**ii
    txt = open("output/D1S32_coefficients_"+str(N)+".txt",'r')
    
    x = np.linspace(-1,1,100)
    y = f(100)